In [1]:
using StanModels
gr(size=(500,500));

ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
#mean_height = mean(df2[:height])
df2[:height_c] = convert(Vector{Float64}, df2[:height]) # .- mean_height
first(df2, 5)

max_height_c = maximum(df2[:height_c])
min_height_c = minimum(df2[:height_c])

heightsmodel = "
data {
  int N;
  real h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=$(min_height_c),upper=$(max_height_c)> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 0.1);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height_c]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.2s.jls", chn)

chn2 = deserialize("m4.2s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StanModels/scripts/04/tmp/heights.stan will be updated.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/04/tmp/heights.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.031, 0.031, 0.031, 0.031) seconds, 0.12 seconds total
Sampling took (0.058, 0.055, 0.050, 0.051) seconds, 0.22 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__            -1301  2.5e-02     1.0  -130

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ mu         │ 177.866 │ 0.100743 │ 0.00159289 │ 0.00184398 │ 4000.0 │
│ 2   │ sigma      │ 24.6259 │ 0.928403 │ 0.0146793  │ 0.0191245  │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 177.677 │ 177.797 │ 177.865 │ 177.934 │ 178.063 │
│ 2   │ sigma      │ 22.8727 │ 24.0007 │ 24.6124 │ 25.2231 │ 26.5189 │


end of m4.2s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*